In [1]:
import numpy

from deap import algorithms
from deap import base
from deap import benchmarks
from deap.benchmarks.tools import hypervolume
from deap import cma
from deap import creator
from deap import tools

In [2]:
# Problem size
N = 5

# ZDT1, ZDT2, DTLZ2
MIN_BOUND = numpy.zeros(N)
MAX_BOUND = numpy.ones(N)
EPS_BOUND = 2.e-5

In [3]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [4]:
def distance(feasible_ind, original_ind):
    """A distance function to the feasibility region."""
    return sum((f - o)**2 for f, o in zip(feasible_ind, original_ind))

a = [1,1,1]
b = [2,2,3]
distance(a, b)

6

In [5]:
def closest_feasible(individual):
    """A function returning a valid individual from an invalid one."""
    feasible_ind = numpy.array(individual)
    feasible_ind = numpy.maximum(MIN_BOUND, feasible_ind)
    feasible_ind = numpy.minimum(MAX_BOUND, feasible_ind)
    return feasible_ind

closest_feasible(1.01)

array([1., 1., 1., 1., 1.])

In [6]:
def valid(individual):
    """Determines if the individual is valid or not."""
    if any(individual < MIN_BOUND) or any(individual > MAX_BOUND):
        return False
    return True

valid(1+1e-6)

False

In [7]:
def close_valid(individual):
    """Determines if the individual is close to valid."""
    if any(individual < MIN_BOUND-EPS_BOUND) or any(individual > MAX_BOUND+EPS_BOUND):
        return False
    return True

close_valid(1+1e-6)

True

In [8]:
# toolbox = base.Toolbox()
# toolbox.register("evaluate", benchmarks.zdt1)
# toolbox.decorate("evaluate", tools.ClosestValidPenalty(valid, closest_feasible, 1.0e+6, distance))

# # toolbox.evaluate([1., 1., 2.])

In [9]:
def sample(individual):
    a1, a2, a3, a4, a5 = individual
    return a1+a2+a3, a4+a5

toolbox = base.Toolbox()
toolbox.register("evaluate", sample)

In [14]:
population = [creator.Individual(x) for x in (numpy.random.uniform(0, 1, (MU, N)))]
population

[[0.465089823110734,
  0.30456880766041416,
  0.4497251420660806,
  0.7861853288384733,
  0.9854887887594566],
 [0.028713546517237787,
  0.11941795297313329,
  0.7346937072895725,
  0.9916367210826488,
  0.49326019444419145],
 [0.6816886144684774,
  0.15130409142721157,
  0.5666068591748589,
  0.9682902956332295,
  0.8860854481030148],
 [0.09352473744891998,
  0.7175323359447847,
  0.3880548753136134,
  0.9855099818530736,
  0.5037618303940624],
 [0.9339830387346554,
  0.5309238209112345,
  0.134830112082454,
  0.993175169301006,
  0.4239075464941564],
 [0.756075566742458,
  0.06614811009002963,
  0.7332091557342566,
  0.4945586149588259,
  0.3754393142764114],
 [0.16459955390261682,
  0.5210766347785348,
  0.900322564934368,
  0.9877501155602212,
  0.5565824227077898],
 [0.42815758363438283,
  0.9742901468336856,
  0.2849545243824819,
  0.6508615890051939,
  0.1862962268514866],
 [0.3886810483004931,
  0.6348927465778288,
  0.8570424565428265,
  0.18514728103227585,
  0.36122621480670

In [10]:
MU, LAMBDA = 10, 10
NGEN = 500
verbose = True
create_plot = False

# The MO-CMA-ES algorithm takes a full population as argument
population = [creator.Individual(x) for x in (numpy.random.uniform(0, 1, (MU, N)))]

for ind in population:
    ind.fitness.values = toolbox.evaluate(ind)

strategy = cma.StrategyMultiObjective(population, sigma=1.0, mu=MU, lambda_=LAMBDA)
toolbox.register("generate", strategy.generate, creator.Individual)
toolbox.register("update", strategy.update)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("min", numpy.min, axis=0)
stats.register("max", numpy.max, axis=0)
   
logbook = tools.Logbook()
logbook.header = ["gen", "nevals"] + (stats.fields if stats else [])

fitness_history = []



for gen in range(NGEN):
        # Generate a new population
        population = toolbox.generate()

        # Evaluate the individuals
        fitnesses = toolbox.map(toolbox.evaluate, population)
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit
            fitness_history.append(fit)
        
        # Update the strategy with the evaluated individuals
        toolbox.update(population)
        
        record = stats.compile(population) if stats is not None else {}
        logbook.record(gen=gen, nevals=len(population), **record)
        if verbose:
            print(logbook.stream)

gen	nevals	min                      	max                    
0  	10    	[-1.28546362 -2.53369704]	[3.49133538 4.14686584]
1  	10    	[-2.28691981 -2.51768913]	[3.52238197 3.37599785]
2  	10    	[-3.13236068 -2.43823103]	[3.48638393 3.73804017]
3  	10    	[-4.06218156 -4.30607593]	[5.18477263 5.27251569]
4  	10    	[-5.68430424 -5.07739515]	[1.60894888 5.07307486]
5  	10    	[-8.51266539 -4.53115849]	[2.02670118 1.54022012]
6  	10    	[-10.14124991  -7.75633616]	[2.10708168 5.35404237]
7  	10    	[-11.9722329   -6.12477117]	[-0.03032856  6.6602529 ]
8  	10    	[-20.23403945 -10.86052259]	[2.77072359 1.25232911]  
9  	10    	[-21.60402839 -14.31947869]	[ 4.68714229 -1.76713354]
10 	10    	[-36.65221428 -20.53937261]	[ 7.71335368 -0.63476972]
11 	10    	[-42.40072226 -32.59467777]	[ 1.12529307 14.3038393 ]
12 	10    	[-60.13450153 -29.0087307 ]	[11.47943852 19.00901466]
13 	10    	[-88.01275877 -29.61965479]	[-7.79348328 48.93811529]
14 	10    	[-130.70565723  -34.19958538]	[-2.8162413  7

175	10    	[-6.50947389e+26 -4.33467858e+26]      	[6.08757547e+25 4.57754136e+26]      
176	10    	[-1.30517210e+27 -5.14651597e+26]      	[2.56642250e+26 1.93868619e+26]      
177	10    	[-1.0378948e+27 -7.7479399e+26]        	[9.52919420e+26 4.00659443e+26]      
178	10    	[-1.70198658e+27 -1.23409929e+27]      	[5.66684881e+26 4.80995552e+26]      
179	10    	[-3.66497078e+27 -8.39672521e+26]      	[-8.73453479e+26  1.05707459e+27]    
180	10    	[-4.55748238e+27 -1.97384996e+27]      	[2.67733436e+26 1.31755145e+27]      
181	10    	[-3.98829864e+27 -1.80455328e+27]      	[-1.98149557e+26  1.07019155e+27]    
182	10    	[-6.50382188e+27 -3.12888559e+27]      	[-4.52605041e+25  1.39829791e+27]    
183	10    	[-5.76025766e+27 -3.51799675e+27]      	[9.09435977e+26 5.86875904e+26]      
184	10    	[-4.82680955e+27 -4.65719175e+27]      	[6.35097193e+26 3.20035379e+26]      
185	10    	[-9.58651718e+27 -5.46366645e+27]      	[4.05211405e+27 5.11951589e+26]      
186	10    	[-1.009542

384	10    	[-7.87400873e+50 -8.91018022e+50]      	[2.74970918e+50 2.63871311e+50]      
385	10    	[-1.50880984e+51 -7.73984642e+50]      	[1.07311403e+51 2.24075618e+50]      
386	10    	[-1.86644264e+51 -1.49599812e+51]      	[8.15806782e+50 8.49209038e+49]      
387	10    	[-2.85728137e+51 -2.09795336e+51]      	[1.23887041e+51 7.77497652e+50]      
388	10    	[-2.40410191e+51 -2.99600508e+51]      	[-7.82735002e+50  3.81447257e+50]    
389	10    	[-4.61918095e+51 -2.98737196e+51]      	[4.47153533e+50 1.11348982e+51]      
390	10    	[-4.60290503e+51 -3.92737052e+51]      	[2.47092190e+51 9.48668483e+50]      
391	10    	[-7.80559285e+51 -5.11939478e+51]      	[ 2.89996653e+51 -5.30594288e+50]    
392	10    	[-9.47895647e+51 -8.38663032e+51]      	[3.35882509e+51 5.96190765e+50]      
393	10    	[-1.17829203e+52 -1.13911575e+52]      	[-7.37608925e+50  3.49363793e+51]    
394	10    	[-1.73379020e+52 -6.81307745e+51]      	[-1.54414528e+51  6.80132779e+51]    
395	10    	[-2.488644

In [11]:
# population = toolbox.generate()
# fitnesses = toolbox.map(toolbox.evaluate, population)

# for ind, fit in zip(population, fitnesses):
#         ind.fitness.values = fit
#         fitness_history.append(fit)

# toolbox.update(population)

In [12]:
# import numpy

# from deap import algorithms
# from deap import base
# from deap import benchmarks
# from deap.benchmarks.tools import hypervolume
# from deap import cma
# from deap import creator
# from deap import tools

# # Problem size
# N = 5

# # ZDT1, ZDT2, DTLZ2
# MIN_BOUND = numpy.zeros(N)
# MAX_BOUND = numpy.ones(N)
# EPS_BOUND = 2.e-5

# # Kursawe
# # MIN_BOUND = numpy.zeros(N) - 5
# # MAX_BOUND = numpy.zeros(N) + 5

# creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
# creator.create("Individual", list, fitness=creator.FitnessMin)

# def distance(feasible_ind, original_ind):
#     """A distance function to the feasibility region."""
#     return sum((f - o)**2 for f, o in zip(feasible_ind, original_ind))

# def closest_feasible(individual):
#     """A function returning a valid individual from an invalid one."""
#     feasible_ind = numpy.array(individual)
#     feasible_ind = numpy.maximum(MIN_BOUND, feasible_ind)
#     feasible_ind = numpy.minimum(MAX_BOUND, feasible_ind)
#     return feasible_ind

# def valid(individual):
#     """Determines if the individual is valid or not."""
#     if any(individual < MIN_BOUND) or any(individual > MAX_BOUND):
#         return False
#     return True

# def close_valid(individual):
#     """Determines if the individual is close to valid."""
#     if any(individual < MIN_BOUND-EPS_BOUND) or any(individual > MAX_BOUND+EPS_BOUND):
#         return False
#     return True

# toolbox = base.Toolbox()
# toolbox.register("evaluate", benchmarks.zdt1)
# toolbox.decorate("evaluate", tools.ClosestValidPenalty(valid, closest_feasible, 1.0e+6, distance))

# def main():
#     # The cma module uses the numpy random number generator
#     # numpy.random.seed(128)

#     MU, LAMBDA = 10, 10
#     NGEN = 500
#     verbose = True
#     create_plot = False

#     # The MO-CMA-ES algorithm takes a full population as argument
#     population = [creator.Individual(x) for x in (numpy.random.uniform(0, 1, (MU, N)))]

#     for ind in population:
#         ind.fitness.values = toolbox.evaluate(ind)

#     strategy = cma.StrategyMultiObjective(population, sigma=1.0, mu=MU, lambda_=LAMBDA)
#     toolbox.register("generate", strategy.generate, creator.Individual)
#     toolbox.register("update", strategy.update)

#     stats = tools.Statistics(lambda ind: ind.fitness.values)
#     stats.register("min", numpy.min, axis=0)
#     stats.register("max", numpy.max, axis=0)
   
#     logbook = tools.Logbook()
#     logbook.header = ["gen", "nevals"] + (stats.fields if stats else [])

#     fitness_history = []

#     for gen in range(NGEN):
#         # Generate a new population
#         population = toolbox.generate()

#         # Evaluate the individuals
#         fitnesses = toolbox.map(toolbox.evaluate, population)
#         for ind, fit in zip(population, fitnesses):
#             ind.fitness.values = fit
#             fitness_history.append(fit)
        
#         # Update the strategy with the evaluated individuals
#         toolbox.update(population)
        
#         record = stats.compile(population) if stats is not None else {}
#         logbook.record(gen=gen, nevals=len(population), **record)
#         if verbose:
#             print(logbook.stream)

#     if verbose:
#         print("Final population hypervolume is %f" % hypervolume(strategy.parents, [11.0, 11.0]))

#         # Note that we use a penalty to guide the search to feasible solutions,
#         # but there is no guarantee that individuals are valid.
#         # We expect the best individuals will be within bounds or very close.
#         num_valid = 0
#         for ind in strategy.parents:
#             dist = distance(closest_feasible(ind), ind)
#             if numpy.isclose(dist, 0.0, rtol=1.e-5, atol=1.e-5):
#                 num_valid += 1
#         print("Number of valid individuals is %d/%d" % (num_valid, len(strategy.parents)))

#         print("Final population:")
#         print(numpy.asarray(strategy.parents))

#     if create_plot:
#         interactive = 0
#         if not interactive:
#             import matplotlib as mpl_tmp
#             mpl_tmp.use('Agg')   # Force matplotlib to not use any Xwindows backend.
#         import matplotlib.pyplot as plt

#         fig = plt.figure()
#         plt.title("Multi-objective minimization via MO-CMA-ES")
#         plt.xlabel("First objective (function) to minimize")
#         plt.ylabel("Second objective (function) to minimize")

#         # Limit the scale because our history values include the penalty.
#         plt.xlim((-0.1, 1.20))
#         plt.ylim((-0.1, 1.20))

#         # Plot all history. Note the values include the penalty.
#         fitness_history = numpy.asarray(fitness_history)
#         plt.scatter(fitness_history[:,0], fitness_history[:,1],
#             facecolors='none', edgecolors="lightblue")

#         valid_front = numpy.array([ind.fitness.values for ind in strategy.parents if close_valid(ind)])
#         invalid_front = numpy.array([ind.fitness.values for ind in strategy.parents if not close_valid(ind)])

#         if len(valid_front) > 0:
#             plt.scatter(valid_front[:,0], valid_front[:,1], c="g")
#         if len(invalid_front) > 0:
#             plt.scatter(invalid_front[:,0], invalid_front[:,1], c="r")

#         if interactive:
#             plt.show()
#         else:
#             print("Writing cma_mo.png")
#             plt.savefig("cma_mo.png")

#     return strategy.parents

# if __name__ == "__main__":
#     solutions = main()